In [ ]:
'''
## Creating Dictionary of Unique Words
en_words=set()
for line in data.encorder:
    for char in line.split():
        if char not in en_words:
            en_words.add(char)
            
de_words=set()
for line in data.decorder:
    for char in line.split():
        if char not in de_words:
            de_words.add(char)
            
# Get lists of words :
input_words = sorted(list(en_words))
target_source_words = sorted(list(de_words))
print('Encorder Vocablary size',len(input_words))
print('Decorder Vocablary size',len(target_source_words))


en_token_to_int_2d = dict()
en_int_to_token_2d = dict()

de_token_to_int_2d = dict()
de_int_to_token_2d= dict()

#Tokenizing the words ( Convert them to numbers ) :
for i,token in enumerate(input_words):
    en_token_to_int_2d[token] = i
    en_int_to_token_2d[i]     = token

for i,token in enumerate(target_source_words):
    de_token_to_int_2d[token] = i
    de_int_to_token_2d[i]     = token
    

    
print('length of en_token_to_int_2d',len(en_token_to_int_2d))
print('length of de_token_to_int_2d',len(de_token_to_int_2d))

de_words_target_3d=set()
for line in data.decorder:
    for char in line[::]:
        if char not in de_words_target_3d:
            de_words_target_3d.add(char)
            
decorder_target_words = sorted(list(de_words_target_3d))


de_token_to_int_3d = dict()
de_int_to_token_3d = dict()

for i,token in enumerate(decorder_target_words):
    de_token_to_int_3d[token] = i
    de_int_to_token_3d[i]     = token
    
print('length of de_token_to_int_3d',len(de_token_to_int_3d))


## Maximum length data.Encorder/data.Decorder
max_len_encorder = list()
for char in range(len(data.encorder)):
    max_len_encorder.append(len(data.encorder[char]))
    
max_len_encorder = max(max_len_encorder)

max_len_decorder = list()
for char in range(len(data.decorder)):
    max_len_decorder.append(len(data.decorder[char]))
    
max_len_decorder = max(max_len_decorder)

print('Maximum Length in data.Encorder',max_len_encorder)
print('Maximum Length in data.decorder',max_len_decorder)



tokenizer = WordPunctTokenizer()
def process_comments(list_sentences):
    comments = []
    for text in (list_sentences):
        text = tokenizer.tokenize(text)
        comments.append(text)
    return comments

input_comments = process_comments(list(data["encorder"].values))
output_comments = process_comments(list(data["decorder"].values))


from keras.preprocessing.sequence import pad_sequences


encoder_input_data = [[en_token_to_int_2d.get(t, 0) for t in comment] 
             for comment in input_comments[:len(input_comments)]]

decoder_input_data = [[de_token_to_int_2d.get(t, 0) for t in comment] 
             for comment in output_comments[:len(output_comments)]]

# padding
encoder_input_data = pad_sequences(encoder_input_data, maxlen=max_len_encorder, 
                     padding="post", truncating="post")

decoder_input_data = pad_sequences(decoder_input_data, maxlen=max_len_decorder, 
                     padding="post", truncating="post")
'''

In [1]:
print('Copied not inspired from below Source')
##https://towardsdatascience.com/word-level-english-to-marathi-neural-machine-translation-using-seq2seq-encoder-decoder-lstm-model-1a913f2dc4a7

Copied not inspired from below Source


In [2]:
import pandas as pd
import numpy as np
import string,re
from string import digits

In [3]:
file_name = '/home/cit/Downloads/fra.txt'
data = pd.read_table(file_name, names=['en', 'de'], encoding='utf-8-sig')

In [4]:
data = data.head(10000)
data.head()

,en,de
0,Go.,Va !
1,Run!,Cours !
2,Run!,Courez !
3,Wow!,Ça alors !
4,Fire!,Au feu !


In [5]:
data.en=data.en.apply(lambda x: x.lower())
data.de=data.de.apply(lambda x: x.lower())

# Process commas :
data.en=data.en.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
data.de=data.de.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

# Getting rid of punctuation
exclude = set(string.punctuation)
data.en=data.en.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
data.de=data.de.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Getting rid of digits
remove_digits = str.maketrans('', '', digits)
data.en=data.en.apply(lambda x: x.translate(remove_digits))
data.de=data.de.apply(lambda x: x.translate(remove_digits))

In [6]:
# Appending SOS andEOS to target data : 
data.de = data.de.apply(lambda x : 'SOS_ '+ x + ' _EOS')

# Create word dictionaries :
en_words=set()
for line in data.en:
    for char in line.split():
        if char not in en_words:
            en_words.add(char)
    
de_words=set()
for line in data.de:
    for char in line.split():
        if char not in de_words:
            de_words.add(char)
            
# get lengths and sizes :
num_en_words = len(en_words)
num_de_words = len(de_words)

max_en_words_per_sample = max([len(sample.split()) for sample in data.en])
max_de_words_per_sample = max([len(sample.split()) for sample in data.de])

num_en_samples = len(data.en)
num_de_samples = len(data.de)

# Get lists of words :
input_words = sorted(list(en_words))
target_words = sorted(list(de_words))

en_token_to_int = dict()
en_int_to_token = dict()

de_token_to_int = dict()
de_int_to_token = dict()

#Tokenizing the words ( Convert them to numbers ) :
for i,token in enumerate(input_words):
    en_token_to_int[token] = i
    en_int_to_token[i]     = token

for i,token in enumerate(target_words):
    de_token_to_int[token] = i
    de_int_to_token[i]     = token

# initiate numpy arrays to hold the data that our seq2seq model will use:
encoder_input_data = np.zeros(
    (num_en_samples, max_en_words_per_sample),
    dtype='float32')
decoder_input_data = np.zeros(
    (num_de_samples, max_de_words_per_sample),
    dtype='float32')
decoder_target_data = np.zeros(
    (num_de_samples, max_de_words_per_sample, num_de_words),
    dtype='float32')

# Process samples, to get input, output, target data:
for i, (input_text, target_text) in enumerate(zip(data.en, data.de)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = en_token_to_int[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = de_token_to_int[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, de_token_to_int[word]] = 1.

In [7]:
encoder_input_data.shape,decoder_input_data.shape,decoder_target_data.shape

((10000, 5), (10000, 13), (10000, 13, 4634))

In [12]:
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout
from keras.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings('ignore')
import gc

In [9]:
print('......Encorder......')
hidden_size = 150
Laten_dim = 64

## Encoprder input and Embedding
encorder_input = Input(shape = (None,),name = 'enc_input')
encorder_embedding_ix = Embedding(len(en_token_to_int), hidden_size,name = 'enc_emb_ix')  #
encorder_embedding =encorder_embedding_ix(encorder_input)

## Layer
encorder_lstm_ix  = LSTM(Laten_dim,return_state=True ,name = 'enc_lstm_ix')
encorder_lstm, state_h, state_c = encorder_lstm_ix(encorder_embedding)

## output 
encoder_states = [state_h, state_c]

......Encorder......


In [10]:
print('......Decorder......')
hidden_size = 150
Laten_dim = 64

## Input and Embedding layers
decorder_input = Input(shape = (None,),name = 'dec_input')
decorder_embedding_ix= Embedding(len(de_token_to_int) ,hidden_size,name = 'dec_emb_ix') #
decorder_embedding = decorder_embedding_ix(decorder_input)

## Decorder layers
decoder_LSTM_layer_ix = LSTM(Laten_dim ,return_sequences=True , return_state=True,name = 'dec_lstm')
decoder_LSTM_layer,_,_ = decoder_LSTM_layer_ix(decorder_embedding , initial_state = encoder_states)

## Decorder output
decorder_dense = Dense(len(de_token_to_int),activation = 'softmax')
decorder_output = decorder_dense(decoder_LSTM_layer)

......Decorder......


In [11]:
model = Model([encorder_input, decorder_input], [decorder_output])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
enc_input (InputLayer)          (None, None)         0                                            
__________________________________________________________________________________________________
dec_input (InputLayer)          (None, None)         0                                            
__________________________________________________________________________________________________
enc_emb_ix (Embedding)          (None, None, 150)    330300      enc_input[0][0]                  
__________________________________________________________________________________________________
dec_emb_ix (Embedding)          (None, None, 150)    695100      dec_input[0][0]                  
____________________________________________________________________________________________

In [13]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=10,
          validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 15s 2ms/step - loss: 1.8685 - accuracy: 0.0763 - val_loss: 1.9817 - val_accuracy: 0.0769
Epoch 2/10
8000/8000 [==============================] - 13s 2ms/step - loss: 1.5571 - accuracy: 0.0936 - val_loss: 1.8821 - val_accuracy: 0.1000
Epoch 3/10
8000/8000 [==============================] - 13s 2ms/step - loss: 1.4357 - accuracy: 0.1062 - val_loss: 1.8240 - val_accuracy: 0.1072
Epoch 4/10
8000/8000 [==============================] - 13s 2ms/step - loss: 1.3645 - accuracy: 0.1114 - val_loss: 1.7753 - val_accuracy: 0.1118
Epoch 5/10
8000/8000 [==============================] - 13s 2ms/step - loss: 1.3116 - accuracy: 0.1160 - val_loss: 1.7408 - val_accuracy: 0.1178
Epoch 6/10
8000/8000 [==============================] - 13s 2ms/step - loss: 1.2640 - accuracy: 0.1233 - val_loss: 1.7154 - val_accu

In [14]:
## Building of Inference Model
encorder_model = Model(encorder_input , encoder_states)

## dECORDER SETUP
decoder_state_input_h = Input(shape=(None,))
decoder_state_input_c = Input(shape=(None,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

## decorder Embedding state
decorder_embedding2 = decorder_embedding_ix(decorder_input)

## To predict the next word in the sequence, set the initial states to the states from the previous time step.
decoder_LSTM_layer2,h2,c2 =  decoder_LSTM_layer_ix(decorder_embedding2 ,initial_state = decoder_states_inputs)
decoder_state2 = [h2,c2]
decorder_output2 = decorder_dense(decoder_LSTM_layer2)

decoder_model = Model(
    [decorder_input] + decoder_states_inputs,
    [decorder_output2] + decoder_state2)

In [15]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in en_token_to_int.items())
reverse_target_char_index = dict(
    (i, char) for char, i in de_token_to_int.items())

In [16]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encorder_model.predict(input_seq) ## h and c state

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0,0] = de_token_to_int['SOS_'] ## \t can use <start also>

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :]) ## Not using Teacher Force 
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_EOS' or len(decoded_sentence) > max_de_words_per_sample):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0,0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [17]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', data.en[seq_index])
    print('Original State',data.de[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: go
Original State SOS_ va  _EOS
Decoded sentence:  prends un maison
-
Input sentence: run
Original State SOS_ cours  _EOS
Decoded sentence:  prends _EOS
-
Input sentence: run
Original State SOS_ courez  _EOS
Decoded sentence:  prends _EOS
-
Input sentence: wow
Original State SOS_ ça alors  _EOS
Decoded sentence:  qui que _EOS
-
Input sentence: fire
Original State SOS_ au feu  _EOS
Decoded sentence:  prends un maison
-
Input sentence: help
Original State SOS_ à laide  _EOS
Decoded sentence:  prends _EOS
-
Input sentence: jump
Original State SOS_ saute _EOS
Decoded sentence:  prends _EOS
-
Input sentence: stop
Original State SOS_ ça suffit  _EOS
Decoded sentence:  arrête _EOS
-
Input sentence: stop
Original State SOS_ stop  _EOS
Decoded sentence:  arrête _EOS
-
Input sentence: stop
Original State SOS_ arrêtetoi  _EOS
Decoded sentence:  arrête _EOS
